In [1]:
# 1. Environment & Imports (with path setup)
import os, time, json, sys
from pathlib import Path

# Determine project root (parent of notebooks directory)
ROOT = Path.cwd().parent  # notebooks -> upreg_classify
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))  # ensure 'src' package import works

# Diagnostic: interpreter path
print('sys.executable:', sys.executable)
# Optional: verify path injection
print('Added to sys.path:', str(ROOT) in sys.path)
print('sys.path[0]:', sys.path[0])

from autogluon.tabular import TabularPredictor
# Import project training entry
from src.train.train import train_and_evaluate

DATA_RAW = ROOT / 'data' / 'raw'
MODELS_DIR = ROOT / 'models'
print('ROOT:', ROOT)
print('Data raw exists:', DATA_RAW.exists())
print('Models dir:', MODELS_DIR)

sys.executable: c:\PythonProjects\rl_reserve_markets\.venv1\Scripts\python.exe
Added to sys.path: True
sys.path[0]: C:\Users\haako\AppData\Local\Programs\Python\Python312\python312.zip
ROOT: c:\PythonProjects\rl_reserve_markets\upreg_classify
Data raw exists: True
Models dir: c:\PythonProjects\rl_reserve_markets\upreg_classify\models
ROOT: c:\PythonProjects\rl_reserve_markets\upreg_classify
Data raw exists: True
Models dir: c:\PythonProjects\rl_reserve_markets\upreg_classify\models


In [2]:
# 2. Utility wrapper
import importlib
from sklearn.metrics import classification_report
from autogluon.tabular import TabularPredictor

def reload_all():
    """Reload key project modules to pick up file changes without restarting VS Code."""
    print('--- Reloading project modules ---')
    import src.data.preprocess as dp
    import src.data.features as feat
    import src.train.hyperparameters as hp
    import src.train.train as tr
    importlib.reload(dp)
    importlib.reload(feat)
    importlib.reload(hp)
    importlib.reload(tr)
    # Re-export key symbols to current namespace if needed
    from src.train.train import train_and_evaluate as _tae
    globals()['train_and_evaluate'] = _tae
    print('Reloaded: preprocess, features, hyperparameters, train')


def run_training(description: str, **kwargs):
    """Run training via train_and_evaluate and report duration + key metrics.
    Required kwargs mirror train_and_evaluate parameters.
    This cell focuses on timing + metrics JSON only.
    """
    print(f'=== Run: {description} ===')
    start = time.time()
    metrics = train_and_evaluate(**kwargs)
    dur = time.time() - start
    print(f'Completed in {dur:.1f}s')
    print(json.dumps(metrics, indent=2))
    return metrics


def show_classification_reports(metrics: dict, output_dir: str):
    """Print validation + test classification reports in a separate cell.
    Call this after run_training(...) and pass its returned metrics and output_dir.
    """
    try:
        label = None
        if isinstance(metrics, dict):
            label = metrics.get('label') or metrics.get('label_col')
        if not (output_dir and label):
            print('Missing output_dir or label in metrics; cannot build reports.')
            return
        pred = TabularPredictor.load(output_dir)
        trainer = None
        try:
            trainer = pred._learner.load_trainer()
        except Exception:
            trainer = None

        # Validation report (if val_data retained)
        if trainer is not None and getattr(trainer, 'val_data', None) is not None:
            val_data = trainer.val_data
            if label in val_data.columns:
                y_true_val = val_data[label]
                X_val = val_data.drop(columns=[label])
                y_pred_val = pred.predict(X_val)
                print('Validation classification report:')
                print(classification_report(y_true_val, y_pred_val))
        else:
            print('No validation data available on trainer; skipping val report.')

        # Test report: use stored test CSV if train_and_evaluate recorded it
        test_csv = None
        if isinstance(metrics, dict):
            test_csv = metrics.get('test_csv') or metrics.get('test_path')
        if test_csv and os.path.exists(test_csv):
            import pandas as pd
            test_df = pd.read_csv(test_csv)
            if label in test_df.columns:
                y_true_test = test_df[label]
                X_test = test_df.drop(columns=[label])
                y_pred_test = pred.predict(X_test)
                print('\nTest classification report:')
                print(classification_report(y_true_test, y_pred_test))
        else:
            if not test_csv:
                print('No test_csv/test_path in metrics; skipping test report.')
            elif not os.path.exists(test_csv):
                print(f'Test CSV path not found: {test_csv}')
    except Exception as e:
        print('Could not generate classification reports:', e)


def base_common(area='NO1'):
    return dict(
        task='multiclass',
        area=area,
        data_dir=str(DATA_RAW),
        include_2024=True,
        heavy_interactions=False,
        dropna=True,
        train_frac=0.6, val_frac=0.2, test_frac=0.2,
        activation_lag_start=4,
        single_persistence=True,
        weight_factor_up=1.0, weight_factor_down=1.0, weight_factor_none=1.0,
        tune_up_bias=True, tune_up_objective='macro',
        num_bag_folds=0, num_stack_levels=0,
        importance_time_limit=60, importance_subsample=1200, importance_top_n=40,
        use_categorical_reglag=False,
        data_start=None,
    )

In [5]:
# 3. Quick RF/XT priority baseline (NO1)
baseline_cfg = base_common(area='NO1')
baseline_cfg.update(dict(
    output_dir=str(MODELS_DIR / 'baseline_rf_xt_priority'),
    time_limit=500,
    presets='best_quality',
    model_preset='rf_xt_priority',
    hpo_trials=0, hpo_searcher='random', hpo_scheduler='local',
))
# Uncomment to run
# run_training('Baseline RF/XT Priority NO1', **baseline_cfg)

In [ ]:
# 4. CatBoost-only + 4 HPO trials (NO1)
cat_hpo_cfg = base_common(area='NO1')
cat_hpo_cfg.update(dict(
    output_dir=str(MODELS_DIR / 'cat_only_hpo4_no_bag'),
    time_limit=500,
    presets='best_quality',
    model_preset='cat_only',
    data_start='2025-03-04',
    hpo_trials=10, hpo_searcher='random', hpo_scheduler='local',
    exclude_persistency_features=True
))
# Uncomment to run
run_training('CatBoost-only HPO(4) NO1', **cat_hpo_cfg)


=== Run: CatBoost-only HPO(4) NO1 ===


In [ ]:
# 5. Baseline RF/XT priority for NO2 (area switch)
no2_baseline_cfg = base_common(area='NO2')
no2_baseline_cfg.update(dict(
    output_dir=str(MODELS_DIR / 'baseline_rf_xt_priority_NO2'),
    time_limit=120,
    presets='best_quality',
    model_preset='rf_xt_priority',
    hpo_trials=0, hpo_searcher='random', hpo_scheduler='local',
))
# Uncomment to run
# run_training('Baseline RF/XT Priority NO2', **no2_baseline_cfg)

In [ ]:
# 6. Stacking example (rf_xt_boost_stack) with modest time limit
stack_cfg = base_common(area='NO1')
stack_cfg.update(dict(
    output_dir=str(MODELS_DIR / 'stack_rf_xt_boost'),
    time_limit=300,
    presets='best_quality',
    model_preset='rf_xt_boost_stack',
    num_bag_folds=0, num_stack_levels=0,  # keep disabled per requirement
    hpo_trials=0,
))
# Uncomment to run
# run_training('Stack Config (no actual bag/stack) NO1', **stack_cfg)

In [ ]:
# 7. Batch runner: choose which configs to execute in sequence
batch = [
    # ('Baseline NO1', baseline_cfg),
    # ('CatBoost HPO4 NO1', cat_hpo_cfg),
    # ('Baseline NO2', no2_baseline_cfg),
]
results = {}
for name, cfg in batch:
    metrics = run_training(name, **cfg)
    results[name] = metrics
if results:
    print('Summary (macro F1):', {k: v.get('val_f1_macro') for k, v in results.items()})

### Notes
- Increase `time_limit` substantially (e.g., 1800+) for higher quality models.
- Set class weighting (e.g., `weight_factor_up=1.3`) to emphasize minority classes.
- Enable categorical RegLag features via `use_categorical_reglag=True` if desired.
- For persistence across re-runs, choose distinct `output_dir` names.

In [ ]:
# XGBoost-only (no HPO) sanity run
xgb_cfg = base_common(area='NO1')
xgb_cfg.update(dict(
    output_dir=str(MODELS_DIR / 'xgb_only_baseline_light'),
    time_limit=180,
    presets='medium_quality',
    model_preset='xgb_only',
    hpo_trials=0,               # disable HPO to isolate training
    hpo_searcher='random',
    hpo_scheduler='local',
    #data_start='2025-06-01',    # keep subset for speed
))
run_training('XGBoost-only BASELINE NO1 LIGHT', **xgb_cfg)


In [ ]:
# 8. XGBoost-only FULL run (fixed variants, full data)
xgb_full_cfg = base_common(area='NO1')
xgb_full_cfg.update(dict(
    output_dir=str(MODELS_DIR / 'xgb_only_full_fixed_actual'),
    time_limit=1800,             # 30 minutes budget; adjust as needed
    presets='best_quality',
    model_preset='xgb_only_fixed',
    hpo_trials=0,                # no HPO; use fixed variants
    hpo_searcher='random',
    hpo_scheduler='local',
    data_start=None,             # ensure full dataset
))
run_training('XGBoost-only FULL FIXED VARIANTS NO1', **xgb_full_cfg)


In [ ]:
# 10. Inspect trained model families from saved predictors
from autogluon.tabular import TabularPredictor
paths = [
    MODELS_DIR / 'xgb_only_full_fixed',
    MODELS_DIR / 'xgb_only_full_fixed_actual',
    MODELS_DIR / 'xgb_only_full_hpo6',
    MODELS_DIR / 'xgb_only_full_hpo2_quick',
]
for p in paths:
    p = str(p)
    if Path(p).exists():
        try:
            pred = TabularPredictor.load(p)
            print(f'Path: {p}')
            print('Models:', pred.model_names())
        except Exception as e:
            print(f'Failed to load {p}:', e)


In [ ]:
# 11. Force-reload training modules to pick up latest hyperparameter presets
import importlib
import src.train.hyperparameters as hp
import src.train.train as tr
hp = importlib.reload(hp)
tr = importlib.reload(tr)
from src.train.hyperparameters import build_hyperparameters
print('xgb_only_fixed keys:', list(build_hyperparameters('xgb_only_fixed', hpo_trials=0).keys()))


In [ ]:
# 12. Sanity: XGBoost-only FIXED on subset, verify model families
xgb_subset_cfg = base_common(area='NO1')
xgb_subset_cfg.update(dict(
    output_dir=str(MODELS_DIR / 'xgb_only_fixed_subset_check'),
    time_limit=1000,
    presets='best_quality',
    model_preset='xgb_only_fixed',
    hpo_trials=5,
    data_start='2025-03-04',
))
metrics = run_training('XGB FIXED SUBSET CHECK', **xgb_subset_cfg)
# Show trained models
pred = TabularPredictor.load(xgb_subset_cfg['output_dir'])
print('Models:', pred.model_names())


In [ ]:
# 13. Sanity: CatBoost-only on subset, mirror cfg of #12
from src.train.train import train_and_evaluate
reload_all()
cat_subset_cfg = base_common(area='NO1')
cat_subset_cfg.update(dict(
    output_dir=str(MODELS_DIR / 'cat_only_subset_check'),
    time_limit=600,
    presets='best_quality',
    model_preset='cat_only',
    hpo_trials=2,
    data_start='2025-03-04',
))
metrics = run_training('CAT ONLY SUBSET CHECK', **cat_subset_cfg)
# Show trained models
pred = TabularPredictor.load(cat_subset_cfg['output_dir'])
print('Models:', pred.model_names())


In [ ]:
# 14. Sanity: LightGBM-only on subset, mirror cfg of #12
lgbm_subset_cfg = base_common(area='NO1')
lgbm_subset_cfg.update(dict(
    output_dir=str(MODELS_DIR / 'lgbm_only_subset_check'),
    time_limit=1000,
    presets='best_quality',
    model_preset='lgbm_only',
    hpo_trials=0,
    data_start='2025-03-04',
))
metrics = run_training('LGBM ONLY SUBSET CHECK', **lgbm_subset_cfg)
# Show trained models
pred = TabularPredictor.load(lgbm_subset_cfg['output_dir'])
print('Models:', pred.model_names())


In [ ]:
# 15. Subset run with persistency interactions disabled (XGB fixed)
xgb_no_inter_subset_cfg = base_common(area='NO1')
xgb_no_inter_subset_cfg.update(dict(
    output_dir=str(MODELS_DIR / 'xgb_fixed_subset_no_interactions'),
    time_limit=600,
    presets='medium_quality',
    model_preset='xgb_only_fixed',
    hpo_trials=0,
    data_start='2025-03-04',
    disable_persistency_interactions=True,
))
metrics = run_training('XGB FIXED SUBSET NO INTERACTIONS', **xgb_no_inter_subset_cfg)
# Show trained models
pred = TabularPredictor.load(xgb_no_inter_subset_cfg['output_dir'])
print('Models:', pred.model_names())


In [ ]:
# 16. LightGBM-only HPO on subset (verify only GBM models)
lgbm_hpo_subset_cfg = base_common(area='NO1')
lgbm_hpo_subset_cfg.update(dict(
    output_dir=str(MODELS_DIR / 'lgbm_only_subset_hpo'),
    time_limit=900,
    presets='best_quality',
    model_preset='lgbm_only',
    hpo_trials=4,
    hpo_searcher='random',
    hpo_scheduler='local',
    data_start='2025-03-04',
))
metrics = run_training('LGBM ONLY HPO(4) SUBSET CHECK', **lgbm_hpo_subset_cfg)
# Show trained models
pred = TabularPredictor.load(lgbm_hpo_subset_cfg['output_dir'])
print('Models:', pred.model_names())

In [ ]:
# 17. Inspect features after persistency-interactions disabled run
# Load predictor from cell #15 output_dir and list any Persistency interaction features
from autogluon.tabular import TabularPredictor
p = TabularPredictor.load(xgb_no_inter_subset_cfg['output_dir'])
feat_names = []
try:
    feat_names = list(p.feature_metadata.get_features())
except Exception:
    try:
        feat_names = list(p._learner.feature_metadata_in.get_features())
    except Exception:
        try:
            feat_names = list(p._learner.features)
        except Exception:
            feat_names = []
print('Total features:', len(feat_names))
bad = [f for f in feat_names if (' x Persistency' in f) or ('Persistency x ' in f) or (' x PersistencyDown' in f) or ('PersistencyDown x ' in f)]
print('Persistency-interaction features found:', bad)